# MONAI Deploy App SDKとMONAI Bundleでデプロイアプリを作成する。

このチュートリアルでは、MONAIで学習し、[MONAI Bundle](https://docs.monai.io/en/latest/bundle_intro.html) フォーマットでパッケージされたPyTorchモデル用の臓器分割アプリケーションを作成する方法を紹介します。

AIモデルの展開には、たとえ研究用であっても臨床画像ネットワークとの統合が必要です。つまり、AIを導入するアプリケーションは、標準ベースの画像プロトコル、特に放射線画像についてはDICOMプロトコルをサポートする必要があります。

通常、DICOMネットワーク通信は、DICOM TCP/IPネットワークプロトコルまたはDICOMWebのいずれかで、DICOMデバイスまたはサービス（例：MONAI Deploy Informatics Gateway）によって処理されるので、展開アプリケーション自体は、入力としてDICOM Part10ファイルを使用して、AI結果をDICOM Part10ファイル（複数）に保存するだけで良いのです。セグメンテーションのユースケースでは、AI結果のDICOMインスタンスファイルはDICOM SegmentationオブジェクトまたはDICOM RT Structure Setであり、分類ではDICOM Structure ReportやDICOM Encapsulated PDFになります。

画像ネットワークと統合され、モダリティやPACS（Picture Archiving and Communications System）からDICOMインスタンスを受け取る場合、AI導入アプリケーションは、複数のシリーズを含むDICOM検査全体を処理しなければならず、その画像の間隔は学習済みモデルによって予想されたものとは異なる可能性があります。MONAI DeployアプリケーションSDKは、このようなケースに一貫して効率的に対応するため、DICOM検査を解析し、アプリケーション定義のルールに従って特定のシリーズを選択し、選択したDICOMシリーズをドメイン固有の画像フォーマットに変換し、適切なDICOM属性を表すメタデータを付加するオペレータと呼ばれるクラスを提供します。その後、画像は前処理段階において、ピクセルデータをテンソルとして推論に使用する前に、間隔、方向、強度などを正規化するためにさらに処理される。

以下のセクションでは、MONAI Deploy アプリケーション SDK を使って MONAI Deploy アプリケーションパッケージを作成する方法と、MONAI Bundle に組み込まれた推論オペレータを使って MONAI Bundle の脾臓 CT Segmentation PyTorch モデルで推論を行う方法を説明します。


:::{note}
DICOM Part 10 ファイルがない場合、3D Slicer などのオープンソースソフトウェアを使用して、NIfTI ファイルを DICOM シリーズに変換することができます。

この例を簡単に実行するために、DICOMファイルと[MONAI Model Zoo](https://github.com/Project-MONAI/model-zoo)で公開されている[Spleen CT Segmentation MONAI Bundle](https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation) をパッケージ化してGoogle Driveで共有しています。

:::

## ApplicationクラスでのOperatorの作成とその接続

5つのOperatorで構成されるアプリケーションを実装します。

- **DICOMDataLoaderOperator**:
    - **Input(dicom_files)**: フォルダパス ([`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html)) **Output(dicom_stiles)**: フォルダパス (`DataPath`) を入力する。
    - **Output(dicom_study_list)**: メモリ上の DICOM study のリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
- **DICOMSeriesSelectorOperator**:
    - **Input(dicom_study_list)**: メモリ上の DICOM スタディのリスト (List[[`DICOMStudy`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DICOMStudy.html)])
    - **Input(selection_rules)**: 選択ルール(Dict)
    - **Output(study_selected_series_list)**: メモリ上の DICOM series オブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
- **DICOMSeriesToVolumeOperator**:
    - **Input(study_selected_series_list)**: メモリ上の DICOM シリーズオブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(image)**: メモリ上の画像オブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **MonaiBundleInferenceOperator**:
    - **Input(image)**: メモリ上のイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Output(pred)**: メモリー上のイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
- **DICOMSegmentationWriterOperator**:
    - **Input(seg_image)**: メモリ上のセグメンテーションイメージオブジェクト ([`Image`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html))
    - **Input(study_selected_series_list)**: メモリー上の DICOM シリーズオブジェクト ([`StudySelectedSeries`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.StudySelectedSeries.html))
    - **Output(dicom_seg_instance)**: ファイルパス（[`DataPath`](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.DataPath.html))


:::{note}
DICOMSegmentationWriterOperator` は、患者の属性と DICOM Study レベルの属性を使用するために、セグメンテーション画像と元の DICOM シリーズのメタデータの両方を必要とします。
:::

アプリケーションのワークフローは以下の通りです。

```{mermaid}
%%{init: {"theme": "base", "themeVariables": { "fontSize": "16px"}} }%%

classDiagram
    direction TB

    DICOMDataLoaderOperator --|> DICOMSeriesSelectorOperator : dicom_study_list...dicom_study_list
    DICOMSeriesSelectorOperator --|> DICOMSeriesToVolumeOperator : study_selected_series_list...study_selected_series_list
    DICOMSeriesToVolumeOperator --|> MonaiBundleInferenceOperator : image...image
    DICOMSeriesSelectorOperator --|> DICOMSegmentationWriterOperator : study_selected_series_list...study_selected_series_list
    MonaiBundleInferenceOperator --|> DICOMSegmentationWriterOperator : pred...seg_image


    class DICOMDataLoaderOperator {
        <in>dicom_files : DISK
        dicom_study_list(out) IN_MEMORY
    }
    class DICOMSeriesSelectorOperator {
        <in>dicom_study_list : IN_MEMORY
        <in>selection_rules : IN_MEMORY
        study_selected_series_list(out) IN_MEMORY
    }
    class DICOMSeriesToVolumeOperator {
        <in>study_selected_series_list : IN_MEMORY
        image(out) IN_MEMORY
    }
    class MonaiBundleInferenceOperator {
        <in>image : IN_MEMORY
        pred(out) IN_MEMORY
    }
    class DICOMSegmentationWriterOperator {
        <in>seg_image : IN_MEMORY
        <in>study_selected_series_list : IN_MEMORY
        dicom_seg_instance(out) DISK
    }
```

### 環境のセットアップ


In [1]:
# MONAIなど、アプリケーションに必要な画像処理パッケージのインストール
!python -c "import monai" || pip install --upgrade -q "monai"
!python -c "import torch" || pip install -q "torch>=1.10.2"
!python -c "import numpy" || pip install -q "numpy>=1.21"
!python -c "import nibabel" || pip install -q "nibabel>=3.2.1"
!python -c "import pydicom" || pip install -q "pydicom>=1.4.2"
!python -c "import highdicom" || pip install -q "highdicom>=0.18.2"
!python -c "import SimpleITK" || pip install -q "SimpleITK>=2.0.0"
!python -c "import typeguard" || pip install -q "typeguard>=2.12.1"

# MONAI Deploy App SDK パッケージのインストール
!python -c "import monai.deploy" || pip install --upgrade -q "monai-deploy-app-sdk"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'monai'
     |████████████████████████████████| 1.1 MB 26.5 MB/s 
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'pydicom'
     |████████████████████████████████| 2.0 MB 10.0 MB/s 
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'highdicom'
     |████████████████████████████████| 800 kB 29.4 MB/s 
     |████████████████████████████████| 3.2 MB 51.6 MB/s 
     |████████████████████████████████| 339 kB 68.2 MB/s 
     |████████████████████████████████| 1.3 MB 41.9 MB/s 
     |████████████████████████████████| 4.3 MB 60.4 MB/s 
Traceback (most recent call last):
  File "<string>", line 1, in <module>
ModuleNotFoundError: No module named 'SimpleITK'
     |████████████████████████████████| 52.8 MB 139 kB/s 
Traceback (most recent call last):
  File "<string

Note: 更新されたパッケージを使用するために、Jupyterカーネルを再起動する必要がある場合があります。

### Google Driveから入力ファイル、モデル・バンドルファイルをダウンロード・抽出

In [2]:
# テストデータおよびMONAIバンドルzipファイルのダウンロード
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ"

# Webブラウザまたはgdownでai_spleen_bundle_dataのzipファイルをダウンロードした後、ai_spleen_bundle_dataのzipファイルをダウンロードしてください。
!unzip -o "ai_spleen_seg_bundle_data.zip"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Downloading...
From: https://drive.google.com/uc?id=1Uds8mEvdGNYUuvFpTtCQ8gNU97bAPCaQ
To: /content/ai_spleen_seg_bundle_data.zip
100% 79.4M/79.4M [00:01<00:00, 49.2MB/s]
Archive:  ai_spleen_seg_bundle_data.zip
  inflating: dcm/1-001.dcm           
  inflating: dcm/1-002.dcm           
  inflating: dcm/1-003.dcm           
  inflating: dcm/1-004.dcm           
  inflating: dcm/1-005.dcm           
  inflating: dcm/1-006.dcm           
  inflating: dcm/1-007.dcm           
  inflating: dcm/1-008.dcm           
  inflating: dcm/1-009.dcm           
  inflating: dcm/1-010.dcm           
  inflating: dcm/1-011.dcm           
  inflating: dcm/1-012.dcm           
  inflating: dcm/1-013.dcm           
  inflating: dcm/1-014.dcm           
  inflating: dcm/1-015.dcm           
  inflating: dcm/1-016.dcm           
  inflating: dcm/1-017.dcm           
  inflating: dcm/1-018.dcm           
  infla

### インポートの設定

ApplicationとOperatorを定義するために必要なクラスやデコレータをインポートしましょう。

In [3]:
import logging

# SegmentDescription属性の設定に必要です。App SDK パッケージに含まれないため、直接インポートする。
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import IOMapping, MonaiBundleInferenceOperator


### モデルバンドル推論オペレータの入力と出力の決定

MONAI Bundleは、基本的にTorchScriptで記述されたPyTorchモデルに、モデルネットワークと処理仕様を記述する追加のメタデータを加えたものです。この演算子は、MONAIユーティリティを使用してMONAI Bundleを解析し、入出力処理と推論に必要なオブジェクトを自動的にインスタンス化するため、MONAIトランスフォーム、インファラー、そしてそれらの依存関係に依存します。

各 Operator クラスは、ベースとなる [Operator](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Operator.html) クラスを継承し、その入出力プロパティは [@input](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.input.html)/[@output](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.output.html) デコレーターで指定することができます。`MonaiBundleInferenceOperator` クラスでは、入力と出力をモデルネットワークと同じ名前とデータ型で定義する必要があります。現在のリリースでは、`IOMapping`オブジェクトを使用して、オペレータの入出力をモデルネットワークの入出力と同じ名前に接続しています。今後のリリースでは、アプリSDKの制限を解除し、自動化する予定です。

Spleen CT Segmentation モデルネットワークは、"image"という名前の入力と、"pred"という名前の出力を持ち、どちらも画像タイプで、すべて App SDK [Image](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.domain.Image.html) にマッピングすることが可能である。この情報の一部は、通常、この[example](https://github.com/Project-MONAI/model-zoo/blob/dev/models/spleen_ct_segmentation/configs/metadata.json)に見られるように、バンドル内のモデルメタデータのnetwork_data_format属性を調べることによって取得されます。

### アプリケーションクラスの作成

アプリケーションクラスは以下のようになります。

[Application](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html) クラスを継承した `App` クラスが定義されています。]

アプリの要件（リソースやパッケージの依存関係）は、[@resource](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.resource.html) と [@env](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.env.html) というデコレータで指定できます。

ベースクラスのメソッドである `compose` はオーバーライドされます。DICOM 解析、シリーズ選択（現在のリリースで最初のシリーズを選択）、ピクセルデータからボリューム画像への変換、セグメンテーションインスタンスの生成に必要なオブジェクトと、モデル固有の `SpleenSegOperator` が生成される。実行パイプラインは、これらのオブジェクトを <a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/modules/_autosummary/monai.deploy.core.Application.html#monai.deploy.core.Application.add_flow">self.add_flow()</a> で連結した Directed Acyclic Graph として作成される。

In [4]:
@resource(cpu=1, gpu=1, memory="7Gi")
# pip_packages には、requirements.txt ファイルへのパス(str)、またはパッケージのリスト を指定します。
# monai pkg はこのクラスでは必要ありません。代わりに、インクルードされている演算子で必要です。
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        logging.info(f"Begin {self.compose.__name__}")

        # SDK 組み込みの演算子と同様に、カスタム演算子を作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # MONAI Bundleをサポートし、推論を自動化する推論オペレータを作成する。
        # IOMappingのラベルは、事前処理と事後処理で入力キーと予測キーに一致する。
        # model_name は、アプリが1つのモデルしか持っていない場合、オプションで指定します。
        # アプリがMAPにパッケージされるとき、オペレータはinit時にバンドルの解析を完了し、
        # バンドルから解析されたオプションのパッケージ情報をパッケージャに提供し、
        # パッケージャがMAPのドッカーイメージにパッケージをインストールできるようにします。
        # 出力IOTypeをDISKに設定することは、リーフオペレータにのみ有効で、この例ではそうではありません。
        #
        # Pertinent MONAI Bundle:
        # https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation

        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
        )

        # 各セグメントに必要なセグメント記述を、実際のアルゴリズムと該当する臓器/組織で提供するDICOM Seg ライターを作成する。
        # segment_label, algorithm_name, algorithm_version は DICOM VR LO タイプで、64 文字以内である。
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.1.0",
            )
        ]
        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            segment_descriptions=segment_descriptions, custom_tags=custom_tags
        )
        # 処理パイプラインを作成し、ソースとデスティネーションの演算子を指定し、
        # 前者の出力が後者の入力と名前と型の両方で一致することを確認します。
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {"pred": "seg_image"})
        # サーフェスメッシュSTL変換演算子を作成し、必要に応じて以下の2行をアンコメントしてアプリの実行フローに追加してください。
        # stl_conversion_op = STLConversionOperator(output_file="stl/spleen.stl")
        # self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {"pred": "image"})

        logging.info(f"End {self.compose.__name__}")


## ローカルでアプリを実行する

Jupyterノートブック上でアプリを実行します。CT AbdomenシリーズのDICOMファイルが `dcm` に、Torch Scriptのモデルが `model.ts` に存在する必要があることに注意してください。あなたの環境での実際のパスを使用してください。


In [5]:
app = AISpleenSegApp()

app.run(input="dcm", output="output", model="model.ts")

Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 75, Operator ID: 508b398c-ffe4-4c1a-bebf-a1260cc57eaf)


[2022-11-01 13:44:27,317] [WARNING] (root) - No selection rules given; select all series.
[2022-11-01 13:44:27,319] [INFO] (root) - Working on study, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
[2022-11-01 13:44:27,320] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239


Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 75, Operator ID: 02dda1e5-c50a-47e2-aa60-d7c53b6500a0)
Working on study, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
Done performing execution of operator DICOMSeriesSelectorOperator

Going to initiate execution of operator DICOMSeriesToVolumeOperator
Executing operator DICOMSeriesToVolumeOperator (Process ID: 75, Operator ID: da291003-7e23-418e-9c7b-0f44bc0314ab)
Done performing execution of operator DICOMSeriesToVolumeOperator

Going to initiate execution of operator MonaiBundleInferenceOperator
Executing operator MonaiBundleInferenceOperator (Process ID: 75, Operator ID: 379fe2c5-92ce-411e-90c4-a1c3282dfe0d)
Done performing execution of operator MonaiBundleInferenc

/usr/local/lib/python3.7/dist-packages/highdicom/valuerep.py:63: UserWarning: The string "C3N-00198" is unlikely to represent the intended person name since it contains only a single component. Construct a person name according to the format in described in http://dicom.nema.org/dicom/2013/output/chtml/part05/sect_6.2.html#sect_6.2.1.2, or, in pydicom 2.2.0 or later, use the pydicom.valuerep.PersonName.from_named_components() method to construct the person name correctly. If a single-component name is really intended, add a trailing caret character to disambiguate the name.
  UserWarning
[2022-11-01 13:44:54,190] [INFO] (highdicom.seg.sop) - add plane #0 for segment #1
[2022-11-01 13:44:54,194] [INFO] (highdicom.seg.sop) - add plane #1 for segment #1
[2022-11-01 13:44:54,200] [INFO] (highdicom.seg.sop) - add plane #2 for segment #1
[2022-11-01 13:44:54,204] [INFO] (highdicom.seg.sop) - add plane #3 for segment #1
[2022-11-01 13:44:54,208] [INFO] (highdicom.seg.sop) - add plane #4 for s

Done performing execution of operator DICOMSegmentationWriterOperator



Jupyter notebook内でアプリケーションが確認できたら、上記のPythonコードをアプリケーションフォルダ内のPythonファイルに書き込んでいきます。

アプリケーションフォルダは以下のような構成になります。

```bash
my_app
├── __main__.py
└── app.py
```

In [6]:
# アプリケーションフォルダーの作成
!mkdir -p my_app

### app.py

In [7]:
%%writefile my_app/app.py

# Copyright 2021-2022 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# あなたは、本使用許諾に従わなかった場合、このファイルを使用することができません。
# 本使用許諾のコピーは、こちらで入手できます。
#     http://www.apache.org/licenses/LICENSE-2.0
# 適用される法律で要求されるか、または書面で合意されない限り、
# 本使用許諾の下で配布されるソフトウェアは、「現状」ベースで配布され、
# 明示または黙示を問わずいかなる種類の保証または条件も付されていません。
# 本使用許諾の下での許可および制限を規定する特定の文言については、本使用許諾を参照してください。


import logging

# SegmentDescription属性の設定に必要です。App SDK パッケージに含まれないため、直接インポートする。
from pydicom.sr.codedict import codes

from monai.deploy.core import Application, resource
from monai.deploy.core.domain import Image
from monai.deploy.core.io_type import IOType
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator, SegmentDescription
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator
from monai.deploy.operators.monai_bundle_inference_operator import IOMapping, MonaiBundleInferenceOperator

# from monai.deploy.operators.stl_conversion_operator import STLConversionOperator  # 必要に応じてインポートしてください。


@resource(cpu=1, gpu=1, memory="7Gi")
# pip_packages には、requirements.txt ファイルへのパス(str)、またはパッケージのリストを指定します。
# monai pkg はこのクラスでは必要なく、代わりに含まれる演算子で必要です。
class AISpleenSegApp(Application):
    def __init__(self, *args, **kwargs):
        """アプリケーションのインスタンスを作成します。"""
        self._logger = logging.getLogger("{}.{}".format(__name__, type(self).__name__))
        super().__init__(*args, **kwargs)

    def run(self, *args, **kwargs):
        # このメソッドは、実行するベースクラスを呼び出す。単に呼び出すだけであれば、省略可能です。
        self._logger.info(f"Begin {self.run.__name__}")
        super().run(*args, **kwargs)
        self._logger.info(f"End {self.run.__name__}")

    def compose(self):
        """アプリ固有の演算子を作成し、処理DAG内で連鎖させる。"""

        logging.info(f"Begin {self.compose.__name__}")

        # SDK 組み込みの演算子と同様に、カスタム演算子を作成します。
        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator(Sample_Rules_Text)
        series_to_vol_op = DICOMSeriesToVolumeOperator()

        # MONAI Bundleをサポートし、推論を自動化する推論オペレータを作成する。
        # IOMappingのラベルは、事前処理と事後処理で入力キーと予測キーに一致する。
        # model_name は、アプリが1つのモデルしか持っていない場合、オプションで指定します。
        # アプリがMAPにパッケージされるとき、オペレータはinit時にバンドルの解析を完了し、
        # バンドルから解析されたオプションのパッケージ情報をパッケージャに提供し、
        # パッケージャがMAPのドッカーイメージにパッケージをインストールできるようにします。
        # 出力IOTypeをDISKに設定することは、リーフオペレータにのみ有効で、この例ではそうではありません。
        #
        # Pertinent MONAI Bundle:
        #   https://github.com/Project-MONAI/model-zoo/tree/dev/models/spleen_ct_segmentation
        bundle_spleen_seg_op = MonaiBundleInferenceOperator(
            input_mapping=[IOMapping("image", Image, IOType.IN_MEMORY)],
            output_mapping=[IOMapping("pred", Image, IOType.IN_MEMORY)],
        )

        # 各セグメントに必要なセグメント記述と実際のアルゴリズムおよび関連する臓器/組織を提供する DICOM Seg ライターを作成します。
        # segment_label, algorithm_name, algorithm_version は DICOM VR LO タイプで、64 文字以内である。
        # https://dicom.nema.org/medical/dicom/current/output/chtml/part05/sect_6.2.html
        segment_descriptions = [
            SegmentDescription(
                segment_label="Spleen",
                segmented_property_category=codes.SCT.Organ,
                segmented_property_type=codes.SCT.Spleen,
                algorithm_name="volumetric (3D) segmentation of the spleen from CT image",
                algorithm_family=codes.DCM.ArtificialIntelligence,
                algorithm_version="0.1.0",
            )
        ]
        custom_tags = {"SeriesDescription": "AI generated Seg, not for clinical use."}

        dicom_seg_writer = DICOMSegmentationWriterOperator(
            segment_descriptions=segment_descriptions, custom_tags=custom_tags
        )

        # 処理パイプラインを作成し、ソースとデスティネーションの演算子を指定し、
        # 前者の出力が後者の入力と名前と型の両方で一致することを確認します。
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(
            series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(series_to_vol_op, bundle_spleen_seg_op, {"image": "image"})
        # dicom_seg_writerは2つの入力を必要とし、それぞれがソースオペレータから来ることに注意してください。
        self.add_flow(
            series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"}
        )
        self.add_flow(bundle_spleen_seg_op, dicom_seg_writer, {"pred": "seg_image"})
        # サーフェスメッシュSTL変換演算子を作成し、必要に応じて以下の2行をアンコメントしてアプリの実行フローに追加してください。
        # stl_conversion_op = STLConversionOperator(output_file="stl/spleen.stl")
        # self.add_flow(bundle_spleen_seg_op, stl_conversion_op, {"pred": "image"})

        logging.info(f"End {self.compose.__name__}")


# これはJSONによるサンプルシリーズ選択ルールで、単純にCTシリーズを選択するものです。
# もし研究が1つ以上のCTシリーズを持っているならば，それら全てが選択されます．
# 詳しくはDICOMSeriesSelectorOperatorを参照して下さい．
Sample_Rules_Text = """
{
    "selections": [
        {
            "name": "CT Series",
            "conditions": {
                "StudyDescription": "(.*?)",
                "Modality": "(?i)CT",
                "SeriesDescription": "(.*?)"
            }
        }
    ]
}
"""

if __name__ == "__main__":
    # アプリを作成し、スタンドアロンでテストします。このモードで実行する場合、以下の点に注意してください。
    #     -m <model file>, for model file path
    #     -i <DICOM folder>, for input DICOM CT series folder
    #     -o <output folder>, for the output folder, default $PWD/output
    # e.g.
    #     monai-deploy exec app.py -i input -m model/model.ts
    #
    logging.basicConfig(level=logging.DEBUG)
    app_instance = AISpleenSegApp(do_run=True)


Writing my_app/app.py


```python
if __name__ == "__main__":
    AISpleenSegApp(do_run=True)
```

上記の行は `python` インタープリタを使ってアプリケーションコードを実行するために必要なものです。

### \_\_main\_\_.py

\_\_main\_\_.py は、<a href="https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html#required-arguments">MONAI Application Packager</a> が、アプリケーションフォルダのパス（例：`python simple_imaging_app`）でアプリケーションを実行したときに、メインアプリケーションコード（`app.py`）を検出するために必要なものです。

In [8]:
%%writefile my_app/__main__.py
from app import AISpleenSegApp

if __name__ == "__main__":
    AISpleenSegApp(do_run=True)

Writing my_app/__main__.py


In [9]:
!ls my_app

app.py	__main__.py


今回は、コマンドラインでアプリを実行してみましょう。

In [10]:
!python my_app -i dcm -o output -m model.ts

2022-11-01 13:46:46,102 - Begin compose
2022-11-01 13:46:46,104 - End compose
2022-11-01 13:46:46,104 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 387, Operator ID: 0a7fd640-f882-495a-827b-fd7d3c061aa9)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 387, Operator ID: 93fb8987-a61e-4ed0-bdb9-5652b7d44f2c)
[2022-11-01 13:46:46,746] [INFO] (root) - Finding series for Selection named: CT Series
[2022-11-01 13:46:46,746] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-11-01 13:46:46,746] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-11-01 13:46:46,746] [INFO] (root) - On attribute: 'StudyDescription' to match value: '

上記のコマンドは、以下のコマンドラインと同じです。

In [11]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'
!monai-deploy exec my_app -i dcm -o output -m model.ts

2022-11-01 13:47:24,286 - Begin compose
2022-11-01 13:47:24,288 - End compose
2022-11-01 13:47:24,288 - Begin run
Going to initiate execution of operator DICOMDataLoaderOperator
Executing operator DICOMDataLoaderOperator (Process ID: 411, Operator ID: 47f55cf4-7792-427d-9937-4120213b8ce3)
Done performing execution of operator DICOMDataLoaderOperator

Going to initiate execution of operator DICOMSeriesSelectorOperator
Executing operator DICOMSeriesSelectorOperator (Process ID: 411, Operator ID: e9db4f83-35cd-485d-80ae-0c245f1bab6e)
[2022-11-01 13:47:24,950] [INFO] (root) - Finding series for Selection named: CT Series
[2022-11-01 13:47:24,951] [INFO] (root) - Searching study, : 1.3.6.1.4.1.14519.5.2.1.7085.2626.822645453932810382886582736291
  # of series: 1
[2022-11-01 13:47:24,951] [INFO] (root) - Working on series, instance UID: 1.3.6.1.4.1.14519.5.2.1.7085.2626.119403521930927333027265674239
[2022-11-01 13:47:24,951] [INFO] (root) - On attribute: 'StudyDescription' to match value: '

In [12]:
!ls output

1.2.826.0.1.3680043.10.511.3.13983115482016230729161876755517780.dcm
1.2.826.0.1.3680043.10.511.3.59096166746055458668374375985689079.dcm
1.2.826.0.1.3680043.10.511.3.99553618575991623147021545637482935.dcm


## Packaging app

[MONAI Application Packager](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/packaging_app.html)でアプリをパッケージ化しよう。

In [13]:
!monai-deploy package -b nvcr.io/nvidia/pytorch:21.11-py3 my_app --tag my_app:latest -m model.ts

[2022-11-01 13:47:58,247] [INFO] (root) - Begin compose
[2022-11-01 13:47:58,248] [INFO] (root) - End compose
Building MONAI Application Package... /bin/sh: 1: docker: not found
Done


:::{note}
MONAIアプリケーションパッケージ(Dockerイメージ)のビルドには時間がかかります。進捗を確認したい場合は、`-l DEBUG`オプションを使用してください。
:::

Dockerイメージが作成されたことが確認できます。

In [14]:
!docker image ls | grep my_app

/bin/bash: docker: command not found


## パッケージ化されたアプリをローカルで実行する

パッケージ化されたアプリは、[MONAI Application Runner](https://docs.monai.io/projects/monai-deploy-app-sdk/en/latest/developing_with_sdk/executing_packaged_app_locally)を使ってローカルに実行することができます。

In [15]:
# コピーしたDICOMファイルは'dcm'フォルダーにあります。

# アプリを起動する
!monai-deploy run my_app:latest dcm output

Checking dependencies...
--> Verifying if "docker" is installed...

ERROR: "docker" not installed, please install docker.
Execution Aborted


In [16]:
!ls output

1.2.826.0.1.3680043.10.511.3.13983115482016230729161876755517780.dcm
1.2.826.0.1.3680043.10.511.3.59096166746055458668374375985689079.dcm
1.2.826.0.1.3680043.10.511.3.99553618575991623147021545637482935.dcm
